# LoC localisation notebook - batch process

This notebook loads a set of images and segments them based on a "segmentation channel" of choice. The structure of the notebook is as follows:

1. Load images
2. Segment
3. Localise segment centroids
4. Unite cell slices over z-stack (equivalent to track over t)
5. Check labelling in Napari

Load necessary Python packages:

In [2]:
import os # this module contains functions for interacting with the operating system (i.e. list files etc)
import glob # good for finding files matching a certain extension
from skimage import io #scikit image data in/out module (for loading/saving images)
import napari # image viewer
import matplotlib.pyplot as plt # figure making module, used to display two images side by side
from tqdm.auto import tqdm # this is a counter that times how long iterative jobs take
import numpy as np # this numerical python module is good for handling images as matrices
import btrack # this is for "tracking" cells through the z-axis
from homuncu_loc import tools # this is for a few custom tools 
import h5py # for creating an empty h5 placeholder file
from macrohet import notify

# print gpu information
!nvcc --version
!nvidia-smi

# load cellpose
from cellpose import core, utils, models, metrics

# check to see if GPU can be used
use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

# define segmentation model parameters
model = models.Cellpose(gpu=use_GPU, 
                        model_type='cyto') # cytoplasmic segmentation 
channels = [0,0] # this means using a grayscale image for both nuclei and cyto channels (even if not using nuclei, still have to say its same colour [greyscale = 0])

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Thu Aug  3 14:15:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:65:00.0  On |                  Off |
| 49%   73C    P2   159W / 300W |   2003MiB / 49140MiB |      2%      Default |
|                               |                      |                  N/A |
+

## 1. Load images

The first step here is to define a base directory where different images for analysis are stored. By defining the path to this directory as a python variable we will reduce the need for long string input of future image paths. 

In [3]:
# take note of root directories on server and local equivalent for saving of dir structure locally
base_dir = server_rootdir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_Mtb area'
# this is the root directory where files will be saved locally
local_rootdir = '/home/dayn/data/homuncu_loc_temp'
print('Here are the experimental data sets contained within base_dir:')
print(os.listdir(base_dir))

Here are the experimental data sets contained within base_dir:
['run2_23-02-104', 'run1_23-01-001_23-01-005']


In [4]:
print('Here are the image data sets contained within the base directory:')
image_file_list = list()

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.tif'):
            image_file_list.append(os.path.join(root, file))

for image_file in image_file_list:
    folder_up = os.path.relpath(os.path.dirname(image_file), base_dir)
    file_name = os.path.basename(image_file)
    print(f'{folder_up}/{file_name}')

Here are the image data sets contained within the base directory:
run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1365.tif
run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1363.tif
run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1361.tif
run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1362.tif
run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1364.tif
run2_23-02-104/2h pi/20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1343.tif
run2_23-02-104/2h pi/20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1342.tif
run2_23-02-104/2h pi/20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1344.tif
run2_23-02-104/2h pi/20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1341.tif
run2_23-02-104/2h pi/20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1340.tif
run1_23-01-001_23-01-005/48h pi/20230707_40X_23-01-005A3_Multichannel Z-Stack_2

Next we will pick some specific images to do, one image per directory to begin with

In [4]:
print('Selecting one image from each directory:')
image_file_list = list()
unique_folders = set()
# collect file names (one per )
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.tif'):
            folder_path = os.path.dirname(os.path.join(root, file))
            if folder_path not in unique_folders:
                unique_folders.add(folder_path)
                image_file_list.append(os.path.join(root, file))
                break  # Move to the next folder after adding one image.

# print file names
for image_file in image_file_list:
    folder_up = os.path.relpath(os.path.dirname(image_file), base_dir)
    file_name = os.path.basename(image_file)
    print(f'{folder_up}/{file_name}')

Selecting one image from each directory:
DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295.tif
DAPI-SPC-PDPN-ZO1/Day14_static/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_246.tif
DAPI-SPC-PDPN-ZO1/Day7_breath/20x_21-12-029A_A3456_Multichannel Z-Stack_20220818_196.tif
DAPI-SPC-PDPN-ZO1/Day7_static/20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_121.tif
DAPI-NKX21-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_286.tif
DAPI-NKX21-PDPN-ZO1/Day14_static/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_235.tif
DAPI-NKX21-PDPN-ZO1/Day7_static/20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_113.tif


Or filter for images you do not want to use

In [5]:
IDs = ['1306', # only 3 z slices  
       '1343', '1342', '1344', '1341' # ZO1 not quite there
      ]

# Use a list comprehension to filter the image_file_list and remove filenames containing faulty IDs
image_file_list = [fn for fn in image_file_list if not any(ID in fn for ID in IDs)]


# Batch process - CURRENTLY REVERSED

This is because I suspect the last items on the list will be the best segmented

In [ ]:
# iterate over file list
for image_file in tqdm(reversed(image_file_list), total = len(image_file_list), desc = 'Iterating over images'):
    # redefine output filename as being h5 file in same directory as image
    output_fn = image_file.replace('.tif', '_z_tracks_masks.h5')
    # relocate output file to a local directory (doesnt like saving to server?)
    output_fn = output_fn.replace(server_rootdir, local_rootdir)
    # create directory structure to hold local file
    os.makedirs(os.path.dirname(output_fn), exist_ok=True)
    # check if output fn exists already, if so then skip
    if os.path.exists(output_fn):
        print(f'Output {os.path.basename(output_fn)} already exists')
        continue
    # create empty placeholder file so that other parallel processes do not start on the same image whilst this is processing
    else:
        # Create an empty HDF5 file with 
        with h5py.File(output_fn, "w") as f:
            pass
    # load image
    try:
        image = io.imread(image_file)
        if image.ndim < 4:
            print('Image is not correct shape')
            continue
    except Exception as e:
        print(f"An error occurred while reading the image: {image_file}")
        print(e)
    # format filenames to print update
    folder_up = os.path.relpath(os.path.dirname(image_file), base_dir)
    file_name = os.path.basename(image_file)
    # print update
    print(f'Loaded image: {folder_up}/{file_name}')
    # set zo1 as mask input channel
    mask_input_channel = image[...,1]
    # check where the cells exist in the image volume, start by defining an empty list to store mean intensity values
    mean_measure = list()
    # iterate over image data set 
    for frame in tqdm(mask_input_channel, total = len(mask_input_channel), 
                      desc = 'Checking where the cells are in the image stack'):
        mean_measure.append(np.mean(frame))    
    # Calculate the average background signal
    average_background = np.mean(mean_measure)
    # Find the indices where the signal crosses above and below the average background
    start_frame = 0 #np.where(mean_measure > average_background)[0][0] - 5 # adding a buffer
    end_frame = len(mask_input_channel) #np.where(mean_measure > average_background)[0][-1] + 5 # adding a buffer 
    ### define empty mask image array (as a list)
    mask_stack = list()
    ### iterate over frames
    for n, frame in tqdm(enumerate(mask_input_channel), total = len(mask_input_channel), 
                         desc = 'Segmenting image stack'):
        if start_frame < n < end_frame:
            ### run segmentation for single frame
            masks, flows, styles, diams = model.eval(frame, diameter=None, flow_threshold=None, channels=channels, min_size = 500)
            
            # trying to fix bug where localisation fails if only one segment present in image
            if np.max(masks) < 2:
                # if beyond the focal range of the stack then just use blank array as masks
                masks = np.zeros(frame.shape, dtype=np.uint16)
                
        else:
            # if beyond the focal range of the stack then just use blank array as masks
            masks = np.zeros(frame.shape, dtype=np.uint16)
        ### append segmentation results to empty to mask image list
        mask_stack.append(masks)
    # turn mask stack into an image array
    mask_stack = np.stack(mask_stack, axis = 0)
    
    # define props 
    props = ('axis_major_length',
             'axis_minor_length',
             'eccentricity',
             'area',
             'orientation',
             'mean_intensity',
             'intensity_image')
    
    # localise all cells in image stack
    objects = btrack.utils.segmentation_to_objects(
                                                   segmentation = mask_stack, # set the masks here 
                                                   intensity_image = image, # provide the image so that the mean intensity can be measured
                                                   properties = props, # provide the cell properties to improve tracker 
                                                   use_weighted_centroid = False, 
    #                                                    assign_class_ID=True,
                                                   )
    # check if mtb infected above threshold
    threshold = 230
    for o in tqdm(objects):
        mtb_glimpse = o.properties['intensity_image'][...,3]
        mtb_status = np.any(mtb_glimpse > threshold)
        mtb_area = np.sum(mtb_glimpse > threshold)
        o.properties['mtb_status'] = mtb_status
        o.properties['mtb_area'] = mtb_area
        del o.properties['intensity_image']
    
    # apply size threshold
    objects = [o for o in objects if o.properties['area'] > 500]
    
    #redefine tuple of properties to remove intensity image
    props = ('axis_major_length',
             'axis_minor_length',
             'eccentricity',
             'area',
             'orientation',
             'mean_intensity',
             )
    
    print(f'{len(objects)} cell objects found in {len(mask_input_channel)} frames/z-slices')
    # track cells over Z
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure('/home/dayn/analysis/btrack/models/particle_config.json')
        ### set max search radius to a very limited radius 
        tracker.max_search_radius = 5
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # use visual features to track
        tracker.features = props
        # append the objects to be tracked
        tracker.append(objects)
        # set the volume
        tracker.volume=((0, mask_input_channel.shape[1]), (0, mask_input_channel.shape[2]), (-1e5, 1e5))
        # track them (in interactive mode)
        tracker.track(step_size=10)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # get the tracks as a python list
        tracks = tracker.tracks
    
    # save out 
    with btrack.io.HDF5FileHandler(output_fn, 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
            writer.write_tracks(tracks)
            writer.write_segmentation(mask_stack)
    # notify me
    notify.send_sms(f'{image_file} complete')

Iterating over images:   0%|          | 0/21 [00:00<?, ?it/s]

Output 20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1314_z_tracks_masks.h5 already exists
Output 20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1320_z_tracks_masks.h5 already exists
Loaded image: run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1315.tif


Checking where the cells are in the image stack:   0%|          | 0/362 [00:00<?, ?it/s]

Segmenting image stack:   0%|          | 0/362 [00:00<?, ?it/s]

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
[INFO][2023/08/03 04:23:18 pm] Localizing objects from segmentation...
03-Aug-23 16:23:18 - btrack.io._localization - INFO     - Localizing objects from segmentation...
100%|█████████████████████████████████████████| 362/362 [01:07<00:00,  5.33it/s]
[INFO][2023/08/03 04:24:26 pm] Objects are of type: <class 'dict'>
03-Aug-23 16:24:26 - btrack.io.utils - INFO     - Objects are of type: <class 'dict'>
[INFO][2023/08/03 04:24:28 pm] ...Found 92561 objects in 362 frames.
03-Aug-23 16:24:28 - btrack.io._localization - INFO     - ...Found 92561 objects in 362 frames.


...

13 cell objects found in 4 frames/z-slices
GLPK Integer Optimizer 5.0
48 rows, 36 columns, 48 non-zeros
36 integer variables, all of which are binary
Preprocessing...
24 rows, 36 columns, 48 non-zeros
36 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 24
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
24 rows, 36 columns, 48 non-zeros
*     0: obj =   1.086432619e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+     0: mip =     not found yet >=              -inf        (1; 0)
+     0: >>>>>   1.086432619e+01 >=   1.086432619e+01   0.0% (1; 0)
+     0: mip =   1.086432619e+01 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND


[INFO][2023/08/03 04:53:51 pm] Closing HDF file: /home/dayn/data/homuncu_loc_temp/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313_z_tracks_masks.h5
03-Aug-23 16:53:51 - btrack.io.hdf - INFO     - Closing HDF file: /home/dayn/data/homuncu_loc_temp/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313_z_tracks_masks.h5


Loaded image: run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1317.tif


Checking where the cells are in the image stack:   0%|          | 0/275 [00:00<?, ?it/s]

Segmenting image stack:   0%|          | 0/275 [00:00<?, ?it/s]

# Check output

In [7]:
v = napari.Viewer()

v.add_image(image, channel_axis = -1, )
v.add_labels(mask_stack)

/home/dayn/analysis/btrack/btrack/dataio.py:3: UserWarning: `btrack.dataio` has been deprecated. Please use `btrack.io` subpackage instead.
  warnings.warn(  # noqa: B028


Assistant skips harvesting pyclesperanto as it's not installed.


<Labels layer 'mask_stack' at 0x7f9bcad0b910>

job1a images/tracks

In [7]:
file_list = [
    "DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295.tif",
    "DAPI-SPC-PDPN-ZO1/Day14_static/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_246.tif",
    "DAPI-SPC-PDPN-ZO1/Day7_breath/20x_21-12-029A_A3456_Multichannel Z-Stack_20220818_196.tif",
    "DAPI-SPC-PDPN-ZO1/Day7_static/20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_121.tif",
    "DAPI-NKX21-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_286.tif",
    "DAPI-NKX21-PDPN-ZO1/Day14_static/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_235.tif",
    "DAPI-NKX21-PDPN-ZO1/Day7_static/20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_113.tif"
]

output_file = "/home/dayn/data/homuncu_loc_temp/job_1a_files.txt"

with open(output_file, "w") as file:
    for item in file_list:
        file.write("%s\n" % item)

print(f"File list saved to {output_file}")


File list saved to /home/dayn/data/homuncu_loc_temp/job_1a_files.txt
